In [2]:
import pickle
import os
import pandas as pd
import pandana as pdn
import numpy as np

import googlemaps
from datetime import datetime
import requests
import time

# Plot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn as sns


# Base utilities
import os

import googlemaps
from datetime import datetime
import requests
import time
import re

# Data Mining
import math
import random
random_state = 42
random.seed(random_state)
seed=random_state
import numpy as np
import pandas as pd
import geopandas as gpd
# import osmnx as ox
import pandana as pdn
import pickle as pkl
import folium

# WKT to Lat and Long

from shapely import wkt
from shapely.geometry import Point

# Plot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn as sns

# Learning
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, train_test_split, KFold

from sklearn.preprocessing import OneHotEncoder

# Models
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors

In [3]:
# Bring trips

trips = pd.read_csv(f"results/trips_pre_model.csv",index_col=0)
replacement_dict = {'Andando': 'Walk'}
trips['Modo'] = trips['Modo'].replace(replacement_dict)

trips_r = pd.read_csv(f"results/trips_pre_model.csv",index_col=0)
replacement_dict = {'Andando': 'Walk'}
trips_r['Modo'] = trips_r['Modo'].replace(replacement_dict)

In [4]:
trips.rename(columns={'Hora_Ini_E': 'Departure time', 'Per_hog': 'HH size', 'Turismos': 'Nº cars/HH',
                   'Sexo': 'Gender', 'Edad': 'Age', 'crnt_tur': 'Driver\'s license',
                   'drive_tt': 'Car $T_t$', 'distance': 'Distance', 'walk_tt': 'Walk $T_t$',
                   'transit_tt_gm': 'PT $T_t$', 'Tipo_familia': 'Family type'}, inplace=True)

mapeo = {1: 2, 2: 1}
trips['Driver\'s license'] = trips['Driver\'s license'].map(mapeo)

trips = trips[['Modo', 'Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Car $T_t$','Distance', 'Walk $T_t$', 'PT $T_t$', 'Family type', 'Group']]

In [5]:
trips_r.rename(columns={'Hora_Ini_E': 'Departure time', 'Per_hog': 'HH size', 'Turismos': 'Nº cars/HH',
                   'Sexo': 'Gender', 'Edad': 'Age', 'crnt_tur': 'Driver\'s license',
                   'drive_tt': 'Car $T_t$', 'distance': 'Distance', 'walk_tt': 'Walk $T_t$',
                   'transit_tt_gm': 'PT $T_t$', 'Tipo_familia': 'Family type'}, inplace=True)

mapeo = {1: 2, 2: 1}
trips_r['Driver\'s license'] = trips_r['Driver\'s license'].map(mapeo)

trips_r = trips_r[['Modo', 'Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Car $T_t$','Distance', 'Walk $T_t$', 'PT $T_t$', 'Family type', 'O_lat', 'O_long', 'Group']]

In [6]:
# Bring model

model_name = "rf_final_1"
file_path = os.path.join("models", f'{model_name}.pkl')

with open(file_path, 'rb') as file:
    model = pkl.load(file)

In [7]:
trips = trips.drop(columns='Modo')

# SCENARIOS

In [359]:
# Predict BL

df = trips.copy()

x = np.array(df.drop(columns='Group'))
y_pred = model.predict(x)
df['y_pred'] = y_pred
# different_rows = (df['Modo'] != df['y_pred']).sum()
# print(different_rows)

# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
df['emissions'] = df.apply(lambda row: (row['Distance']/1000) * multipliers[row['y_pred']], axis=1)
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
df['distance_weekly'] = df['Distance'] * 5
kms = df.groupby('y_pred')['distance_weekly'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).tolist() # *2/500 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

[54.4, 20.4, 25.2]
0.57
[717720, 125050, 26290]


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Car combustion 162

Car electric 88

Bus combustion 91

Bus electric 42

Train 38

# REMOTE WORK

In [12]:
df = trips.copy()

x = np.array(df.drop(columns='Group'))
y_pred = model.predict(x)
df['y_pred'] = y_pred
# different_rows = (df['Modo'] != df['y_pred']).sum()
# print(different_rows)

# 30% remote working
n = len(df)
num_ones = int(n * 0.3)
values = [1] * num_ones + [0] * (n - num_ones)
np.random.shuffle(values)
df['RW'] = values

days = 3 # working days

# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
multipliers_rw = {'Car': 162*days/5, 'PT': 91*days/5, 'Walk': 0*days/5}
df['emissions'] = df.apply(
    lambda row: (row['Distance'] / 1000) * (multipliers_rw[row['y_pred']] if row['RW'] == 1 else multipliers[row['y_pred']]),
    axis=1
)
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
df['distance_weekly'] = df.apply(
    lambda row: row['Distance'] * days if row['RW'] == 1 else row['Distance'] * 5,
    axis=1
)
kms = df.groupby('y_pred')['distance_weekly'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).tolist() # *2/1000 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

[54.4, 20.4, 25.2]
0.5
[632370, 110180, 23040]


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


# PT ENHANCEMENT

In [15]:
df = trips.copy()

# Change PT enhancement
df['PT $T_t$'] = df['PT $T_t$'] * 0.65
df['Car $T_t$'] = df['Car $T_t$'] * 1

x = np.array(df.drop(columns='Group'))
y_pred = model.predict(x)
df['y_pred'] = y_pred
# different_rows = (df['Modo'] != df['y_pred']).sum()
# print(different_rows)

# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
df['emissions'] = df.apply(lambda row: (row['Distance']/1000) * multipliers[row['y_pred']], axis=1)
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
kms = df.groupby('y_pred')['Distance'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).mul(5).tolist() # *2/500 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

[47.8, 26.0, 26.1]
0.55
[656900, 185170, 26990]


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


# DESTINATIONS RELOCATION

In [9]:
# DRIVE

networks = dict.fromkeys({
 "walk",
 "drive",
})

for k in networks:
    print(k)
    networks[k] = pdn.network.Network.from_hdf5(f'../input_data/networks/{k}_net.h5')

# # TRANSIT

transit = dict.fromkeys({
 "transit_0001",
 "transit_0102",
 "transit_0203", # walk_network because transit is not available
 "transit_0304", # walk_network because transit is not available
 "transit_0405", # walk_network because transit is not available
 "transit_0506",
 "transit_0607",
 "transit_0708",
 "transit_0809",
 "transit_0910",
 "transit_1011",
 "transit_1112",
 "transit_1213",
 "transit_1314",
 "transit_1415",
 "transit_1516",
 "transit_1617",
 "transit_1718",
 "transit_1819",
 "transit_1920",
 "transit_2021",
 "transit_2122",
 "transit_2223",
 "transit_2324",
})

for k in transit:
    # print(k)
    transit[k] = pdn.network.Network.from_hdf5(f'../input_data/transit_together_24h/{k}.h5')

walk
drive


In [10]:
destinations = pd.read_csv('../input_data/destinations.csv',index_col=0)

In [21]:
# 20 MINUTE PT REGION

def travel_time(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    distance = distance*1000
    return distance

df = trips_r.copy()

df['Relocation'] = 0
percent = 25
n = len(df)
A = int(n * percent / 100)
filtro = df['PT $T_t$'] > 0
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'Relocation'] = 1

# Change destination
df['D_lat'] = np.nan
df['D_long'] = np.nan

for index, row in df.iterrows():
    if row['Relocation'] == 1:
        distances = travel_time(row['O_lat'], row['O_long'], destinations['D_lat'], destinations['D_long'])
        
        within_range = destinations[(distances <= 3000) & (distances >= 1000)]

        if not within_range.empty:
            chosen = within_range.sample(n=1)
            df.at[index, 'D_lat'] = chosen['D_lat'].values[0]
            df.at[index, 'D_long'] = chosen['D_long'].values[0]

# RECALCULATE TRIPS VARIABLES

df['drive_tt'] = np.nan
df['distance'] = np.nan
df['walk_tt'] = np.nan
df['transit_tt_gm'] = np.nan

df_reloc_1 = df[df['Relocation'] == 1]
df_reloc_0 = df[df['Relocation'] == 0]

df_reloc_1 = df_reloc_1.dropna(subset=['D_lat', 'D_long']).reset_index(drop=True)

df_reloc_1["drive_tt"] = networks['drive'].shortest_path_lengths(
                    networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
                    networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
                    imp_name='drive_time_s'
                    )

df_reloc_1["drive_tt"] = df_reloc_1["drive_tt"] / 60 # To min

df_reloc_1["distance"] = networks['drive'].shortest_path_lengths(
            networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
            imp_name='distance'
            )

df_reloc_1["walk_tt"] = networks['walk'].shortest_path_lengths(
            networks['walk'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['walk'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
            )
df_reloc_1["walk_tt"] = df_reloc_1["walk_tt"] / 60

# Add TRANSIT

for k in transit:
    df_reloc_1[f"{k}_tt"] = transit[k].shortest_path_lengths(
            transit[k].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            transit[k].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
    )

def asignar_valor(row):
    if 1 <= row['Departure time'] <= 12:
        return row['transit_0001_tt']
    elif 13 <= row['Departure time'] <= 24:
        return row['transit_0102_tt']
    elif 25 <= row['Departure time'] <= 36:
        return row['transit_0203_tt']
    elif 37 <= row['Departure time'] <= 48:
        return row['transit_0304_tt']
    elif 49 <= row['Departure time'] <= 60:
        return row['transit_0405_tt']
    elif 61 <= row['Departure time'] <= 72:
        return row['transit_0506_tt']
    elif 73 <= row['Departure time'] <= 84:
        return row['transit_0607_tt']
    elif 85 <= row['Departure time'] <= 96:
        return row['transit_0708_tt']
    elif 97 <= row['Departure time'] <= 108:
        return row['transit_0809_tt']
    elif 109 <= row['Departure time'] <= 120:
        return row['transit_0910_tt']
    elif 121 <= row['Departure time'] <= 132:
        return row['transit_1011_tt']
    elif 133 <= row['Departure time'] <= 144:
        return row['transit_1112_tt']
    elif 145 <= row['Departure time'] <= 156:
        return row['transit_1213_tt']
    elif 157 <= row['Departure time'] <= 168:
        return row['transit_1314_tt']
    elif 169 <= row['Departure time'] <= 180:
        return row['transit_1415_tt']
    elif 181 <= row['Departure time'] <= 192:
        return row['transit_1516_tt']
    elif 193 <= row['Departure time'] <= 204:
        return row['transit_1617_tt']
    elif 205 <= row['Departure time'] <= 216:
        return row['transit_1718_tt']
    elif 217 <= row['Departure time'] <= 228:
        return row['transit_1819_tt']
    elif 229 <= row['Departure time'] <= 240:
        return row['transit_1920_tt']
    elif 241 <= row['Departure time'] <= 252:
        return row['transit_2021_tt']
    elif 253 <= row['Departure time'] <= 264:
        return row['transit_2122_tt']
    elif 265 <= row['Departure time'] <= 276:
        return row['transit_2223_tt']
    elif 277 <= row['Departure time'] <= 288:
        return row['transit_2324_tt']
    else:
        return None 

# Create new column
df_reloc_1['transit_tt_gm'] = df_reloc_1.apply(asignar_valor, axis=1)

# Don't know why but there are some extreme outliers on drive_tt. 70000 mins?
df_reloc_1 = df_reloc_1.loc[df_reloc_1['drive_tt'] < 200].reset_index(drop=True)

df_reloc_1 = df_reloc_1.drop(columns=['transit_1314_tt',
    'transit_2122_tt', 'transit_1617_tt', 'transit_0405_tt',
    'transit_0001_tt', 'transit_1213_tt', 'transit_1415_tt',
    'transit_0910_tt', 'transit_0708_tt', 'transit_1011_tt',
    'transit_0102_tt', 'transit_1516_tt', 'transit_0506_tt',
    'transit_2021_tt', 'transit_2324_tt', 'transit_1819_tt',
    'transit_0203_tt', 'transit_0809_tt', 'transit_1718_tt',
    'transit_2223_tt', 'transit_0304_tt', 'transit_1920_tt',
    'transit_0607_tt', 'transit_1112_tt'])

df_reloc_1['Car $T_t$'] = df_reloc_1['drive_tt']
df_reloc_1['Distance'] = df_reloc_1['distance']
df_reloc_1['PT $T_t$'] = df_reloc_1['transit_tt_gm']
df_reloc_1['Walk $T_t$'] = df_reloc_1['walk_tt']

df = pd.concat([df_reloc_1, df_reloc_0], ignore_index=True)

df = df[['Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Car $T_t$','Distance', 'Walk $T_t$', 'PT $T_t$', 'Family type', 'Relocation', 'Group']]

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

x = np.array(df.drop(columns=['Relocation', 'Group']))
y_pred = model.predict(x)
df['y_pred'] = y_pred

# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
df['emissions'] = df.apply(lambda row: (row['Distance']/1000) * multipliers[row['y_pred']], axis=1)
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
kms = df.groupby('y_pred')['Distance'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).mul(5).tolist() # *2/500 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(10829722475, 2433423935), (11185574908, 2433423935), (487897104, 2433423935)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(10829722475, 2433423935), (11185574908, 2433423935), (487897104, 2433423935)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(11294818636, 11185664641), (10065375813, 11185664641), (2420360452, 1118

[48.7, 27.5, 23.8]
0.46
[559860, 127940, 27620]


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(44707, 109077), (19240, 109077), (9265, 109077), (23577, 109077), (9235, 109077), (9127, 109077), (116486, 109077)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(40580, 104950), (15113, 104950), (5138, 104950), (19450, 104950), (5108, 104950), (5000, 104950), (112359, 104950)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [375]:
# 15 MINUTE WALKABLE

def travel_time(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    distance = distance*1000
    return distance

df = trips_r.copy()

df['Relocation'] = 0
percent = 40
n = len(df)
A = int(n * percent / 100)
filtro = df['PT $T_t$'] > 25
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'Relocation'] = 1

# Change destination
df['D_lat'] = np.nan
df['D_long'] = np.nan

def generar_punto_dentro_circulo(centro_lat, centro_long, radio):
    angulo = np.random.uniform(0, 2 * np.pi)
    distancia = np.sqrt(np.random.uniform(0, 1)) * radio
    delta_lat = distancia * np.cos(angulo)
    delta_long = distancia * np.sin(angulo)
    punto_lat = centro_lat + delta_lat
    punto_long = centro_long + delta_long
    return punto_lat, punto_long

radio = 0.005

for index, row in df.iterrows():
    if row['Relocation'] == 1:
        lat, long = generar_punto_dentro_circulo(row['O_lat'], row['O_long'], radio)
        df.at[index, 'D_lat'] = lat
        df.at[index, 'D_long'] = long

# RECALCULATE TRIPS VARIABLES

df['drive_tt'] = np.nan
df['distance'] = np.nan
df['walk_tt'] = np.nan
df['transit_tt_gm'] = np.nan

df_reloc_1 = df[df['Relocation'] == 1]
df_reloc_0 = df[df['Relocation'] == 0]

df_reloc_1 = df_reloc_1.dropna(subset=['D_lat', 'D_long']).reset_index(drop=True)

df_reloc_1["drive_tt"] = networks['drive'].shortest_path_lengths(
                    networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
                    networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
                    imp_name='drive_time_s'
                    )

df_reloc_1["drive_tt"] = df_reloc_1["drive_tt"] / 60 # To min

df_reloc_1["distance"] = networks['drive'].shortest_path_lengths(
            networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
            imp_name='distance'
            )

df_reloc_1["walk_tt"] = networks['walk'].shortest_path_lengths(
            networks['walk'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['walk'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
            )
df_reloc_1["walk_tt"] = df_reloc_1["walk_tt"] / 60

# Add TRANSIT

for k in transit:
    df_reloc_1[f"{k}_tt"] = transit[k].shortest_path_lengths(
            transit[k].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            transit[k].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
    )

def asignar_valor(row):
    if 1 <= row['Departure time'] <= 12:
        return row['transit_0001_tt']
    elif 13 <= row['Departure time'] <= 24:
        return row['transit_0102_tt']
    elif 25 <= row['Departure time'] <= 36:
        return row['transit_0203_tt']
    elif 37 <= row['Departure time'] <= 48:
        return row['transit_0304_tt']
    elif 49 <= row['Departure time'] <= 60:
        return row['transit_0405_tt']
    elif 61 <= row['Departure time'] <= 72:
        return row['transit_0506_tt']
    elif 73 <= row['Departure time'] <= 84:
        return row['transit_0607_tt']
    elif 85 <= row['Departure time'] <= 96:
        return row['transit_0708_tt']
    elif 97 <= row['Departure time'] <= 108:
        return row['transit_0809_tt']
    elif 109 <= row['Departure time'] <= 120:
        return row['transit_0910_tt']
    elif 121 <= row['Departure time'] <= 132:
        return row['transit_1011_tt']
    elif 133 <= row['Departure time'] <= 144:
        return row['transit_1112_tt']
    elif 145 <= row['Departure time'] <= 156:
        return row['transit_1213_tt']
    elif 157 <= row['Departure time'] <= 168:
        return row['transit_1314_tt']
    elif 169 <= row['Departure time'] <= 180:
        return row['transit_1415_tt']
    elif 181 <= row['Departure time'] <= 192:
        return row['transit_1516_tt']
    elif 193 <= row['Departure time'] <= 204:
        return row['transit_1617_tt']
    elif 205 <= row['Departure time'] <= 216:
        return row['transit_1718_tt']
    elif 217 <= row['Departure time'] <= 228:
        return row['transit_1819_tt']
    elif 229 <= row['Departure time'] <= 240:
        return row['transit_1920_tt']
    elif 241 <= row['Departure time'] <= 252:
        return row['transit_2021_tt']
    elif 253 <= row['Departure time'] <= 264:
        return row['transit_2122_tt']
    elif 265 <= row['Departure time'] <= 276:
        return row['transit_2223_tt']
    elif 277 <= row['Departure time'] <= 288:
        return row['transit_2324_tt']
    else:
        return None 

# Create new column
df_reloc_1['transit_tt_gm'] = df_reloc_1.apply(asignar_valor, axis=1)

# Don't know why but there are some extreme outliers on drive_tt. 70000 mins?
df_reloc_1 = df_reloc_1.loc[df_reloc_1['drive_tt'] < 200].reset_index(drop=True)

df_reloc_1 = df_reloc_1.drop(columns=['transit_1314_tt',
    'transit_2122_tt', 'transit_1617_tt', 'transit_0405_tt',
    'transit_0001_tt', 'transit_1213_tt', 'transit_1415_tt',
    'transit_0910_tt', 'transit_0708_tt', 'transit_1011_tt',
    'transit_0102_tt', 'transit_1516_tt', 'transit_0506_tt',
    'transit_2021_tt', 'transit_2324_tt', 'transit_1819_tt',
    'transit_0203_tt', 'transit_0809_tt', 'transit_1718_tt',
    'transit_2223_tt', 'transit_0304_tt', 'transit_1920_tt',
    'transit_0607_tt', 'transit_1112_tt'])

df_reloc_1['Car $T_t$'] = df_reloc_1['drive_tt']
df_reloc_1['Distance'] = df_reloc_1['distance']
df_reloc_1['PT $T_t$'] = df_reloc_1['transit_tt_gm']
df_reloc_1['Walk $T_t$'] = df_reloc_1['walk_tt']

df = pd.concat([df_reloc_1, df_reloc_0], ignore_index=True)

df = df[['Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Car $T_t$','Distance', 'Walk $T_t$', 'PT $T_t$', 'Family type', 'Relocation', 'Group']]

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

x = np.array(df.drop(columns=['Relocation', 'Group']))
y_pred = model.predict(x)
df['y_pred'] = y_pred

# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
df['emissions'] = df.apply(lambda row: (row['Distance']/1000) * multipliers[row['y_pred']], axis=1)
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
kms = df.groupby('y_pred')['Distance'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).mul(5).tolist() # *2/500 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(2297916503, 2297916855), (488057921, 9704972827), (310910969, 2297916855), (2300635290, 2297916917)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(2297916503, 2297916855), (488057921, 9704972827), (310910969, 2297916855), (2300635290, 2297916917)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(1411635, 2495674088), (331

[32.8, 13.4, 53.8]
0.34
[428400, 76660, 39650]


In [ ]:
def travel_time(lat1, lon1, lat2, lon2):

    lat1_series = pd.Series([lat1], index=[0])
    lon1_series = pd.Series([lon1], index=[0])
    lat2_series = pd.Series([lat2], index=[0])
    lon2_series = pd.Series([lon2], index=[0])

    # Obtener node IDs
    origin_ids = transit["transit_0910"].get_node_ids(lon1_series, lat1_series)
    destination_ids = transit["transit_0910"].get_node_ids(lon2_series, lat2_series)

    # Calcular tiempos de viaje entre los node IDs de origen y destino
    travel_time = transit["transit_0910"].shortest_path_lengths(origin_ids, destination_ids)

    return travel_time 






df = trips_r.copy()

# df = df.sample(n=100)

# Choose the workers that change destination.
# Just to the ones with PT T_t > 25 mins
# percent = 10
# n = len(df)
# num_ones = int(n * percent / 100)
# values = [1] * num_ones + [0] * (n - num_ones)
# np.random.shuffle(values)
# df['Relocation'] = values

df['Relocation'] = 0
percent = 10
n = len(df)
A = int(n * percent / 100)
print(A)

# Suponiendo que 'df' ya está definido y tiene las columnas adecuadas
# Primero, filtramos las filas donde "PT T_t" es mayor que 25
filtro = df['PT $T_t$'] > 25

# Ahora, queremos poner el valor 1 en la columna 'Relocation' para las primeras 'A' filas que cumplan con la condición
# Utilizamos head(A) para seleccionar las primeras 'A' filas después del filtro
indices_a_modificar = df.loc[filtro].index

# Ahora asignamos el valor 1 a esas filas en la columna 'Relocation'
df.loc[indices_a_modificar, 'Relocation'] = 1

# Change destination
df['D_lat'] = np.nan
df['D_long'] = np.nan

count = 0

for index, row in df.iterrows():
    if row['Relocation'] == 1.0:
        print(count)
        count +=1
        destinations['travel_time'] = np.nan
        destinations = destinations.sample(frac=1).reset_index(drop=True)
        for index2, row2 in destinations.iterrows():
            tt = travel_time(row['O_lat'], row['O_long'], row2['D_lat'], row2['D_long'])
            if (12 < tt < 18):
                df.at[index, 'D_lat'] = row2['D_lat']
                df.at[index, 'D_long'] = row2['D_long']
                break


# x = np.array(df)
# y_pred = model.predict(x)
# df['y_pred'] = y_pred
# # different_rows = (df['Modo'] != df['y_pred']).sum()
# # print(different_rows)

# # SHARE
# percentages = df['y_pred'].value_counts(normalize=True) * 100
# percentages_array = percentages.tolist()
# ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
# ordered_percentages = [round(num, 1) for num in ordered_percentages]
# print(ordered_percentages)

# # CO2
# multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
# df['emissions'] = df.apply(lambda row: (row['Distance']/1000) * multipliers[row['y_pred']], axis=1)
# co2 = df['emissions'].sum()
# co2 = co2/1000000 # from grams to tones
# co2 =  co2*230*2/len(df)
# print(co2)

# # KMS
# kms = df.groupby('y_pred')['Distance'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).mul(5).tolist() # *2/500 (2 trips per day and transform from meters to kms).
# kms = [int(round(x, -1)) for x in kms]
# print(kms)

# EVS

In [378]:
df = trips.copy()

x = np.array(df.drop(columns='Group'))
y_pred = model.predict(x)
df['y_pred'] = y_pred

# 30% ev
n = len(df)
num_ones = int(n * 0.3)
values = [1] * num_ones + [0] * (n - num_ones)
np.random.shuffle(values)
df['EV'] = values

df['EV'] = 0
percent = 50
n = len(df)
A = int(n * percent / 100)
filtro = df['y_pred'].isin(['PT', 'Car'])
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'EV'] = 1

# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
multipliers_ev = {'Car': 88, 'PT': 42, 'Walk': 0}
df['emissions'] = df.apply(
    lambda row: (row['Distance'] / 1000) * (multipliers_ev[row['y_pred']] if row['EV'] == 1 else multipliers[row['y_pred']]),
    axis=1
)
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
kms = df.groupby('y_pred')['Distance'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).mul(5).tolist() # *2/1000 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

[54.4, 20.4, 25.2]
0.44
[717720, 125050, 26290]


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


# COMBINED 1

In [24]:
# 20 MINUTE PT REGION

def travel_time(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    distance = distance*1000
    return distance

df = trips_r.copy()

df['Relocation'] = 0
percent = 40
n = len(df)
A = int(n * percent / 100)
filtro = df['PT $T_t$'] > 25
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'Relocation'] = 1

# Change destination
df['D_lat'] = np.nan
df['D_long'] = np.nan

for index, row in df.iterrows():
    if row['Relocation'] == 1:
        distances = travel_time(row['O_lat'], row['O_long'], destinations['D_lat'], destinations['D_long'])
        
        within_range = destinations[(distances <= 3000) & (distances >= 1000)]

        if not within_range.empty:
            chosen = within_range.sample(n=1)
            df.at[index, 'D_lat'] = chosen['D_lat'].values[0]
            df.at[index, 'D_long'] = chosen['D_long'].values[0]

# RECALCULATE TRIPS VARIABLES

df['drive_tt'] = np.nan
df['distance'] = np.nan
df['walk_tt'] = np.nan
df['transit_tt_gm'] = np.nan

df_reloc_1 = df[df['Relocation'] == 1]
df_reloc_0 = df[df['Relocation'] == 0]

df_reloc_1 = df_reloc_1.dropna(subset=['D_lat', 'D_long']).reset_index(drop=True)

df_reloc_1["drive_tt"] = networks['drive'].shortest_path_lengths(
                    networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
                    networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
                    imp_name='drive_time_s'
                    )

df_reloc_1["drive_tt"] = df_reloc_1["drive_tt"] / 60 # To min

df_reloc_1["distance"] = networks['drive'].shortest_path_lengths(
            networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
            imp_name='distance'
            )

df_reloc_1["walk_tt"] = networks['walk'].shortest_path_lengths(
            networks['walk'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['walk'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
            )
df_reloc_1["walk_tt"] = df_reloc_1["walk_tt"] / 60

# Add TRANSIT

for k in transit:
    df_reloc_1[f"{k}_tt"] = transit[k].shortest_path_lengths(
            transit[k].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            transit[k].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
    )

def asignar_valor(row):
    if 1 <= row['Departure time'] <= 12:
        return row['transit_0001_tt']
    elif 13 <= row['Departure time'] <= 24:
        return row['transit_0102_tt']
    elif 25 <= row['Departure time'] <= 36:
        return row['transit_0203_tt']
    elif 37 <= row['Departure time'] <= 48:
        return row['transit_0304_tt']
    elif 49 <= row['Departure time'] <= 60:
        return row['transit_0405_tt']
    elif 61 <= row['Departure time'] <= 72:
        return row['transit_0506_tt']
    elif 73 <= row['Departure time'] <= 84:
        return row['transit_0607_tt']
    elif 85 <= row['Departure time'] <= 96:
        return row['transit_0708_tt']
    elif 97 <= row['Departure time'] <= 108:
        return row['transit_0809_tt']
    elif 109 <= row['Departure time'] <= 120:
        return row['transit_0910_tt']
    elif 121 <= row['Departure time'] <= 132:
        return row['transit_1011_tt']
    elif 133 <= row['Departure time'] <= 144:
        return row['transit_1112_tt']
    elif 145 <= row['Departure time'] <= 156:
        return row['transit_1213_tt']
    elif 157 <= row['Departure time'] <= 168:
        return row['transit_1314_tt']
    elif 169 <= row['Departure time'] <= 180:
        return row['transit_1415_tt']
    elif 181 <= row['Departure time'] <= 192:
        return row['transit_1516_tt']
    elif 193 <= row['Departure time'] <= 204:
        return row['transit_1617_tt']
    elif 205 <= row['Departure time'] <= 216:
        return row['transit_1718_tt']
    elif 217 <= row['Departure time'] <= 228:
        return row['transit_1819_tt']
    elif 229 <= row['Departure time'] <= 240:
        return row['transit_1920_tt']
    elif 241 <= row['Departure time'] <= 252:
        return row['transit_2021_tt']
    elif 253 <= row['Departure time'] <= 264:
        return row['transit_2122_tt']
    elif 265 <= row['Departure time'] <= 276:
        return row['transit_2223_tt']
    elif 277 <= row['Departure time'] <= 288:
        return row['transit_2324_tt']
    else:
        return None 

# Create new column
df_reloc_1['transit_tt_gm'] = df_reloc_1.apply(asignar_valor, axis=1)

# Don't know why but there are some extreme outliers on drive_tt. 70000 mins?
df_reloc_1 = df_reloc_1.loc[df_reloc_1['drive_tt'] < 200].reset_index(drop=True)

df_reloc_1 = df_reloc_1.drop(columns=['transit_1314_tt',
    'transit_2122_tt', 'transit_1617_tt', 'transit_0405_tt',
    'transit_0001_tt', 'transit_1213_tt', 'transit_1415_tt',
    'transit_0910_tt', 'transit_0708_tt', 'transit_1011_tt',
    'transit_0102_tt', 'transit_1516_tt', 'transit_0506_tt',
    'transit_2021_tt', 'transit_2324_tt', 'transit_1819_tt',
    'transit_0203_tt', 'transit_0809_tt', 'transit_1718_tt',
    'transit_2223_tt', 'transit_0304_tt', 'transit_1920_tt',
    'transit_0607_tt', 'transit_1112_tt'])

df_reloc_1['Car $T_t$'] = df_reloc_1['drive_tt']
df_reloc_1['Distance'] = df_reloc_1['distance']
df_reloc_1['PT $T_t$'] = df_reloc_1['transit_tt_gm']
df_reloc_1['Walk $T_t$'] = df_reloc_1['walk_tt']

df = pd.concat([df_reloc_1, df_reloc_0], ignore_index=True)

df = df[['Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Car $T_t$','Distance', 'Walk $T_t$', 'PT $T_t$', 'Family type', 'Group']]

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(2433579603, 2433423935), (487896846, 2433423935), (9502904086, 2433423935), (9502904086, 2433423935), (487896816, 2433423935), (5186010938, 2433423935), (5186010945, 2433423935)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(2433579603, 2433423935), (487896846, 2433423935), (9502904086, 2433423935), (9502904086, 2433423935), (487896816, 2433423935), (5186010938, 2433423935), (5186010945, 2433423935)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsign

[45.7, 31.5, 22.8]
0.39
[471400, 129460, 28090]


In [27]:
# Change PT enhancement
df['PT $T_t$'] = df['PT $T_t$'] * 0.65

[39.1, 37.2, 23.7]
0.38
[424080, 175450, 29410]


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [30]:
x = np.array(df.drop(columns='Group'))
y_pred = model.predict(x)
df['y_pred'] = y_pred

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [403]:
# % remote working
n = len(df)
num_ones = int(n * 0.3)
values = [1] * num_ones + [0] * (n - num_ones)
np.random.shuffle(values)
df['RW'] = values

days = 3

# 30% ev
n = len(df)
num_ones = int(n * 0.3)
values = [1] * num_ones + [0] * (n - num_ones)
np.random.shuffle(values)
df['EV'] = values

df['EV'] = 0
percent = 25
n = len(df)
A = int(n * percent / 100)
filtro = df['y_pred'].isin(['PT', 'Car'])
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'EV'] = 1

In [404]:
# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
multipliers_rw = {'Car': 162*days/5, 'PT': 91*days/5, 'Walk': 0*days/5}
multipliers_ev = {'Car': 88, 'PT': 42, 'Walk': 0} # g co2/km
multipliers_rw_ev = {'Car': 88*days/5, 'PT': 42*days/5, 'Walk': 0*days/5}

df['emissions'] = 0
for index, row in df.iterrows():
    if row['RW'] == 1:
        if row['EV'] == 1:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers_rw_ev[row['y_pred']]
        else:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers_rw[row['y_pred']]
    else:
        if row['EV'] == 1:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers_ev[row['y_pred']]
        else:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers[row['y_pred']]
            
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
df['distance_weekly'] = df.apply(
    lambda row: row['Distance'] * days if row['RW'] == 1 else row['Distance'] * 5,
    axis=1
)
kms = df.groupby('y_pred')['distance_weekly'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).tolist() # *2/1000 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

[39.0, 36.6, 24.4]
0.29
[371410, 152600, 26720]


# COMBINED 2

In [416]:
# 15 MINUTE WALKABLE

def travel_time(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    distance = distance*1000
    return distance

df = trips_r.copy()

df['Relocation'] = 0
percent = 25
n = len(df)
A = int(n * percent / 100)
filtro = df['PT $T_t$'] > 25
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'Relocation'] = 1

# Change destination
df['D_lat'] = np.nan
df['D_long'] = np.nan

def generar_punto_dentro_circulo(centro_lat, centro_long, radio):
    angulo = np.random.uniform(0, 2 * np.pi)
    distancia = np.sqrt(np.random.uniform(0, 1)) * radio
    delta_lat = distancia * np.cos(angulo)
    delta_long = distancia * np.sin(angulo)
    punto_lat = centro_lat + delta_lat
    punto_long = centro_long + delta_long
    return punto_lat, punto_long

radio = 0.005

for index, row in df.iterrows():
    if row['Relocation'] == 1:
        lat, long = generar_punto_dentro_circulo(row['O_lat'], row['O_long'], radio)
        df.at[index, 'D_lat'] = lat
        df.at[index, 'D_long'] = long

# RECALCULATE TRIPS VARIABLES

df['drive_tt'] = np.nan
df['distance'] = np.nan
df['walk_tt'] = np.nan
df['transit_tt_gm'] = np.nan

df_reloc_1 = df[df['Relocation'] == 1]
df_reloc_0 = df[df['Relocation'] == 0]

df_reloc_1 = df_reloc_1.dropna(subset=['D_lat', 'D_long']).reset_index(drop=True)

df_reloc_1["drive_tt"] = networks['drive'].shortest_path_lengths(
                    networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
                    networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
                    imp_name='drive_time_s'
                    )

df_reloc_1["drive_tt"] = df_reloc_1["drive_tt"] / 60 # To min

df_reloc_1["distance"] = networks['drive'].shortest_path_lengths(
            networks['drive'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['drive'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat),
            imp_name='distance'
            )

df_reloc_1["walk_tt"] = networks['walk'].shortest_path_lengths(
            networks['walk'].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            networks['walk'].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
            )
df_reloc_1["walk_tt"] = df_reloc_1["walk_tt"] / 60

# Add TRANSIT

for k in transit:
    df_reloc_1[f"{k}_tt"] = transit[k].shortest_path_lengths(
            transit[k].get_node_ids(df_reloc_1.O_long,df_reloc_1.O_lat),
            transit[k].get_node_ids(df_reloc_1.D_long,df_reloc_1.D_lat)
    )

def asignar_valor(row):
    if 1 <= row['Departure time'] <= 12:
        return row['transit_0001_tt']
    elif 13 <= row['Departure time'] <= 24:
        return row['transit_0102_tt']
    elif 25 <= row['Departure time'] <= 36:
        return row['transit_0203_tt']
    elif 37 <= row['Departure time'] <= 48:
        return row['transit_0304_tt']
    elif 49 <= row['Departure time'] <= 60:
        return row['transit_0405_tt']
    elif 61 <= row['Departure time'] <= 72:
        return row['transit_0506_tt']
    elif 73 <= row['Departure time'] <= 84:
        return row['transit_0607_tt']
    elif 85 <= row['Departure time'] <= 96:
        return row['transit_0708_tt']
    elif 97 <= row['Departure time'] <= 108:
        return row['transit_0809_tt']
    elif 109 <= row['Departure time'] <= 120:
        return row['transit_0910_tt']
    elif 121 <= row['Departure time'] <= 132:
        return row['transit_1011_tt']
    elif 133 <= row['Departure time'] <= 144:
        return row['transit_1112_tt']
    elif 145 <= row['Departure time'] <= 156:
        return row['transit_1213_tt']
    elif 157 <= row['Departure time'] <= 168:
        return row['transit_1314_tt']
    elif 169 <= row['Departure time'] <= 180:
        return row['transit_1415_tt']
    elif 181 <= row['Departure time'] <= 192:
        return row['transit_1516_tt']
    elif 193 <= row['Departure time'] <= 204:
        return row['transit_1617_tt']
    elif 205 <= row['Departure time'] <= 216:
        return row['transit_1718_tt']
    elif 217 <= row['Departure time'] <= 228:
        return row['transit_1819_tt']
    elif 229 <= row['Departure time'] <= 240:
        return row['transit_1920_tt']
    elif 241 <= row['Departure time'] <= 252:
        return row['transit_2021_tt']
    elif 253 <= row['Departure time'] <= 264:
        return row['transit_2122_tt']
    elif 265 <= row['Departure time'] <= 276:
        return row['transit_2223_tt']
    elif 277 <= row['Departure time'] <= 288:
        return row['transit_2324_tt']
    else:
        return None 

# Create new column
df_reloc_1['transit_tt_gm'] = df_reloc_1.apply(asignar_valor, axis=1)

# Don't know why but there are some extreme outliers on drive_tt. 70000 mins?
df_reloc_1 = df_reloc_1.loc[df_reloc_1['drive_tt'] < 200].reset_index(drop=True)

df_reloc_1 = df_reloc_1.drop(columns=['transit_1314_tt',
    'transit_2122_tt', 'transit_1617_tt', 'transit_0405_tt',
    'transit_0001_tt', 'transit_1213_tt', 'transit_1415_tt',
    'transit_0910_tt', 'transit_0708_tt', 'transit_1011_tt',
    'transit_0102_tt', 'transit_1516_tt', 'transit_0506_tt',
    'transit_2021_tt', 'transit_2324_tt', 'transit_1819_tt',
    'transit_0203_tt', 'transit_0809_tt', 'transit_1718_tt',
    'transit_2223_tt', 'transit_0304_tt', 'transit_1920_tt',
    'transit_0607_tt', 'transit_1112_tt'])

df_reloc_1['Car $T_t$'] = df_reloc_1['drive_tt']
df_reloc_1['Distance'] = df_reloc_1['distance']
df_reloc_1['PT $T_t$'] = df_reloc_1['transit_tt_gm']
df_reloc_1['Walk $T_t$'] = df_reloc_1['walk_tt']

df = pd.concat([df_reloc_1, df_reloc_0], ignore_index=True)

df = df[['Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Car $T_t$','Distance', 'Walk $T_t$', 'PT $T_t$', 'Family type', 'Group']]

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(2297916503, 2297916928), (657781173, 9659586529)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(2297916503, 2297916928), (657781173, 9659586529)]
  warnings.warn(
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(1327938140, 8235607148), (1328294438, 8235607157), (455544341, 4705353770), (266889867, 2495674087), (1845691927, 1735775257), (

In [417]:
# Change PT enhancement
df['PT $T_t$'] = df['PT $T_t$'] * 0.8

In [418]:
x = np.array(df.drop(columns='Group'))
y_pred = model.predict(x)
df['y_pred'] = y_pred

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [419]:
# % remote working
n = len(df)
num_ones = int(n * 0.3)
values = [1] * num_ones + [0] * (n - num_ones)
np.random.shuffle(values)
df['RW'] = values

days = 1 # working days

# % ev
n = len(df)
num_ones = int(n * 0.3)
values = [1] * num_ones + [0] * (n - num_ones)
np.random.shuffle(values)
df['EV'] = values

df['EV'] = 0
percent = 50
n = len(df)
A = int(n * percent / 100)
filtro = df['y_pred'].isin(['PT', 'Car'])
indices_a_modificar = df.sample(n=A, random_state=42).index
df.loc[indices_a_modificar, 'EV'] = 1

In [420]:
# SHARE
percentages = df['y_pred'].value_counts(normalize=True) * 100
percentages_array = percentages.tolist()
ordered_percentages = percentages.reindex(['Car', 'PT', 'Walk']).fillna(0).tolist()
ordered_percentages = [round(num, 1) for num in ordered_percentages]
print(ordered_percentages)

# CO2
multipliers = {'Car': 162, 'PT': 91, 'Walk': 0} # g co2/km
multipliers_rw = {'Car': 162*days/5, 'PT': 91*days/5, 'Walk': 0*days/5}
multipliers_ev = {'Car': 88, 'PT': 42, 'Walk': 0} # g co2/km
multipliers_rw_ev = {'Car': 88*days/5, 'PT': 42*days/5, 'Walk': 0*days/5}

df['emissions'] = 0
for index, row in df.iterrows():
    if row['RW'] == 1:
        if row['EV'] == 1:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers_rw_ev[row['y_pred']]
        else:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers_rw[row['y_pred']]
    else:
        if row['EV'] == 1:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers_ev[row['y_pred']]
        else:
            df.loc[index, 'emissions'] = (row['Distance']/1000) * multipliers[row['y_pred']]
            
co2 = df['emissions'].sum()
co2 = co2/1000000 # from grams to tones
co2 =  round(co2*230*2/len(df), 2)
print(co2)

# KMS
df['distance_weekly'] = df.apply(
    lambda row: row['Distance'] * days if row['RW'] == 1 else row['Distance'] * 5,
    axis=1
)
kms = df.groupby('y_pred')['distance_weekly'].sum().reindex(['Car', 'PT', 'Walk']).fillna(0).div(500).tolist() # *2/1000 (2 trips per day and transform from meters to kms).
kms = [int(round(x, -1)) for x in kms]
print(kms)

[38.5, 18.1, 43.3]
0.24
[387560, 85900, 26460]
